In [1]:
from tensorflow.keras.datasets import mnist
import numpy as np

(x_train, _), (x_test, _) = mnist.load_data()


x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

x_train = x_train.reshape((len(x_train), 28, 28, 1))
x_test = x_test.reshape((len(x_test), 28, 28, 1))

11493376/11490434 [==============================] - 3s 0us/step


In [2]:
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K

In [3]:
input_image = Input(shape=(28, 28, 1)) # Channels last

# Encoder
x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_image)
x = MaxPooling2D((2, 2), padding='same')(x)

x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)

x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

# Decoder
x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)

x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)

x = Conv2D(16, (3, 3), activation='relu')(x)
x = UpSampling2D((2, 2))(x)

decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = Model(input_image, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

autoencoder.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 28, 28, 16)        160       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 8)         1160      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 8)           0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 8)           584       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 8)           0     

In [4]:
from tensorflow.keras.callbacks import TensorBoard

In [6]:
autoencoder.fit(x_train, x_train,
                epochs=50,
                batch_size=128,
                shuffle=True,
                validation_data=(x_test, x_test),
                callbacks=[TensorBoard(log_dir='tf/notebooks/tf_logs')])

Epoch 1/50
469/469 [==============================] - 81s 172ms/step - loss: 0.1101 - val_loss: 0.1077
Epoch 2/50
469/469 [==============================] - 81s 173ms/step - loss: 0.1078 - val_loss: 0.1055
Epoch 3/50
469/469 [==============================] - 77s 164ms/step - loss: 0.1061 - val_loss: 0.1040
Epoch 4/50
469/469 [==============================] - 76s 162ms/step - loss: 0.1046 - val_loss: 0.1035
Epoch 5/50
469/469 [==============================] - 62s 132ms/step - loss: 0.1034 - val_loss: 0.1017
Epoch 6/50
469/469 [==============================] - 59s 126ms/step - loss: 0.1022 - val_loss: 0.1007
Epoch 7/50
469/469 [==============================] - 74s 157ms/step - loss: 0.1013 - val_loss: 0.0996
Epoch 8/50
469/469 [==============================] - 69s 148ms/step - loss: 0.1003 - val_loss: 0.0989
Epoch 9/50
469/469 [==============================] - 77s 163ms/step - loss: 0.0995 - val_loss: 0.0980
Epoch 10/50
139/469 [=======>......................] - ETA: 1:10 - loss: 

KeyboardInterrupt: 